In [1]:

import cv2
import numpy as np

In [2]:
CLASSES = ['background', 'aeroplane','bicycle', 'bird','boat','bottle','bus','car',
           'chair','cow','diningtable', 'dog', 'horse','motobike',
           'pottedplant', 'person', 'sheep',  'sofa', 'trian', 'tvmonitor']

COLORS = np.random.uniform(0,255,size = (len(CLASSES),3)) 

In [3]:
net = cv2.dnn.readNetFromCaffe('./deploy.prototxt',
                               'mobilenet_iter_73000.caffemodel')

In [4]:
cap = cv2.VideoCapture(0)

done = True

if not cap.isOpened():
    print("file not opened")
    done = False

In [5]:
delay = int(1000/ cap.get(cv2.CAP_PROP_FPS))

In [6]:
while done:
    ret, image = cap.read()
    if ret :
        (h,w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image,(300,300)),
                                     0.007843,
                                     (300,300),
                                     127.5)
        net.setInput(blob)
        detections = net.forward()
        
        for i in np.arange(0,detections.shape[2]):
            confidence = detections[0,0,i,2]
            if confidence > 0.2 :
                idx = int(detections[0,0,i,1])
                box = detections[0,0,i,3:7] *np.array([w,h,w,h])
                (sx,sy,ex,ey) = box.astype('int')
                
                label = '{} : {} %'.format(CLASSES[idx],confidence *100)
                cv2.rectangle(image,(sx,sy),(ex,ey), COLORS[idx],2)
                y = sy -15 if sy -15 > 15 else sy +15
                cv2.putText(image,
                            label,
                            (sx,y),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            COLORS[idx],
                            2)
        cv2.imshow('Detection',image)
        key = cv2.waitKey(delay)
        if key == ord('q'):
            break
cap.release
cv2.destroyAllWindows()
        
        